In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib import style
style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.options.display.max_columns = 100

from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split,RandomizedSearchCV,KFold,StratifiedKFold
import lightgbm as lgb
import xgboost as xgb

In [ ]:
train = pd.read_csv("../input/janatahack-machine-learning-for-banking/train_fNxu4vz.csv")
test  = pd.read_csv("../input/janatahack-machine-learning-for-banking/test_fjtUOL8.csv")
Submission = test[['Loan_ID']]
test.drop("Loan_ID",axis=1,inplace=True)
train.drop("Loan_ID",axis=1,inplace=True)
print (train.shape, test.shape)

In [ ]:
train.info()

In [ ]:
test.info()

1. There are missing values in Length_Employed, Home_Owner, Annual_Income, and Months_Since_Delinquency variables in both Train and Test datasets. 
2. The dataset has a mix of Object, Float and int variables.
3. Dependent variable is categorical in nature with 3 levels. 

In [ ]:
train.head()

In [ ]:
test.head()

### EDA

In [ ]:
train['Loan_Amount_Requested'] = train['Loan_Amount_Requested'].apply(lambda x: x.replace(",","")).astype(float)
test['Loan_Amount_Requested'] = test['Loan_Amount_Requested'].apply(lambda x: x.replace(",","")).astype(float)

In [ ]:
train['Loan_Amount_Requested'].describe()

In [ ]:
Q1 = train['Loan_Amount_Requested'].quantile(0.25)
Q3 = train['Loan_Amount_Requested'].quantile(0.75)
IQR = Q3-Q1
Lower_Range = Q1-1.5*IQR
Upper_Range = Q3+1.5*IQR
print ("Values Below {} and Values above {} are considered as outliers in Loan_Amount_Requested Variable".format(Lower_Range,Upper_Range))

In [ ]:
sns.distplot(train['Loan_Amount_Requested'])

1. We cannot make out anything from the distribution above, although it might be considered tending towards Positively skewed distribution. 
2. The minimum value is 500 and Maximum is 35000, so the range is 34500.
3. The median is 12075 with a standard deviation of 8281. 

In [ ]:
train['Length_Employed'].nunique()

In [ ]:
train['Length_Employed'].value_counts(dropna=False).plot(kind='bar')
# There are lot of missing values
# Most of them have been employed for more than 10 years. 

In [ ]:
train['Home_Owner'].unique()

In [ ]:
train['Home_Owner'].value_counts(dropna=False)

In [ ]:
train['Home_Owner'].value_counts(dropna=False).plot(kind='bar')
# There are lot of missing values
# Most of them have been employed for more than 10 years. 

In [ ]:
# Since the value count of Other and None are negligible as compared to the size of the dataset, they can be combined with any
# other value.

In [ ]:
train['Annual_Income'].describe()

In [ ]:
sns.distplot(np.log1p(train['Annual_Income'].dropna()),kde=False)

In [ ]:
train.loc[train['Annual_Income']>1000000,'Annual_Income'] = train['Annual_Income'].mean()

In [ ]:
test.loc[test['Annual_Income']>1000000,'Annual_Income'] = test['Annual_Income'].mean()

1. As we can see in the above plot, a lot of income variables are between 4000 and 100000.
2. There seem to be some outliers that need to be explored.
3. Outliers are the reason for high mean.
4. There are a few people whose incomes are more than a million.
5. We can also see that Log transformed Annual_Income looks perfectly normal.

In [ ]:
train[train['Annual_Income'] >= 100000]

In [ ]:
Q1 = train['Annual_Income'].quantile(0.25)
Q3 = train['Annual_Income'].quantile(0.75)
IQR = Q3-Q1
Lower_Range = Q1-1.5*IQR
Upper_Range = Q3+1.5*IQR
print ("Values Below {} and Values above {} are considered as outliers in Annual_Income Variable".format(Lower_Range,Upper_Range))

In [ ]:
train['Income_Verified'].unique()

In [ ]:
train['Income_Verified'].value_counts(dropna=False).plot(kind='bar')
# There are no missing values here, but a large portion of incomes are not verified. 
# There are also significant observations where the income source is verified but not the income.

In [ ]:
train['Purpose_Of_Loan'].unique()

In [ ]:
train['Purpose_Of_Loan'].value_counts(dropna=False)

In [ ]:
train['Purpose_Of_Loan'].value_counts(dropna=False).plot(kind='bar')

1. Debt_Consolidation and Credit_Card seem to be the common reasons for loan application.

In [ ]:
train['Debt_To_Income'].describe()

In [ ]:
sns.distplot(train['Debt_To_Income'])
# Seems to be normally distributed. 

1. As per the description of the variable, it is 'A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested loan, divided by the borrower’s self-reported monthly income.'
2. A value of 0 in Debt_to_Income ratio seems strange. 

In [ ]:
train['Inquiries_Last_6Mo'].unique()

In [ ]:
train['Inquiries_Last_6Mo'].value_counts(dropna=False)

In [ ]:
train['Inquiries_Last_6Mo'].value_counts(dropna=False).plot(kind='bar')
# Most of the borrowers have not made any inquiries in the last 6 months. There are a few who have made 8 inquiries as well.
# No missing values in this variable.

In [ ]:
train['Months_Since_Deliquency'].describe()

1. More than 50% missing values in this variable. We have to check how useful this variable is.
2. One way is , if the Months_Since_Deliquency is low, then the interest charged might be higher and vice versa.

In [ ]:
train['Number_Open_Accounts'].describe()

In [ ]:
train['Number_Open_Accounts'].nunique()

In [ ]:
train['Number_Open_Accounts'].value_counts().head(10)

1. Most of the people have more than 6 open accounts currently.

In [ ]:
train['Total_Accounts'].describe()

In [ ]:
train['Total_Accounts'].value_counts().head(10)

In [ ]:
train[['Number_Open_Accounts','Total_Accounts']]

1. We can find the number of accounts closed by the customers. 

In [ ]:
train['Gender'].value_counts(dropna=False).plot(kind='bar')
# No missing values here.
# Male population is greater than Female.

### Dependent Variable

In [ ]:
train['Interest_Rate'].value_counts()

In [ ]:
train['Interest_Rate'].value_counts().plot(kind='bar')

1. Interest Rate 2 seems to be more popular. Followed by 3 and 1.
2. Around 50% of them are applicable for Interest Rate category 2.

### Bi-Variate Analysis

In [ ]:
train.groupby('Length_Employed')['Loan_Amount_Requested'].agg(['count','min','max','mean','median','std'])

1. We can see that the average Loan Amount Requested is maximum for people with 10+ years of employment.
2. The maximum value of the Loan_Amount_Requested is same for all levels of employment although minimum amount differs.
3. The average Loan_Amount_Requested is minimum for people with <1 year of being employed. 

In [ ]:
train.groupby('Home_Owner')['Loan_Amount_Requested'].agg(['count','min','max','mean','median','std'])

1. We can see the average Loan_Amount_Requested is maximum for people with Home_Owner status as Mortgage.

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(x=train['Loan_Amount_Requested'],y=train['Annual_Income'])

1. We can see the points at the top of the plot that although the Annual_Income is high, the Loan_Amount_Requested is low.
2. We can also see a few Annual_Incomes that are more than 1000000.

In [ ]:
train.groupby('Purpose_Of_Loan')['Loan_Amount_Requested'].agg(['count','min','max','mean','median','std'])

1. As we can see from the table above, the average loan amount requested is highest for small business followed by debt_consolidation.
2. Average income requested is also high for House loans. 

In [ ]:
train.groupby('Interest_Rate')['Loan_Amount_Requested'].agg(['count','min','max','mean','median','std'])

1. Clearly, there is a distinction between the average loan amount requested for the type of interest rates.
2. Average Loan Amount seems to be high for Interest Rate category 3 and there is not much between the loan amount for categories 1 and 2. 

In [ ]:
train.groupby('Interest_Rate')['Annual_Income'].agg(['count','min','max','mean','median','std'])

1. The mean Annual_Income for Interest Rate Category 1 is the highest. And it is almost the same for Interest Rate categories 2 and 3. 

### Feature Generation, Feature Extraction, etc ..

In [ ]:
for df in [train,test]:
    # Filling Missing Values
    df['Length_Employed'].fillna("Missing",inplace=True)
    df['Home_Owner'].fillna("Missing",inplace=True)
    df['Home_Owner'].replace({"Other":"New","None":"New"},inplace=True) # Combining Other and None to a single Category
    df['Annual_Income'].fillna(df['Annual_Income'].mean(),inplace=True) # Filling missing value with median income
    df['Deliquency_Status'] = df['Months_Since_Deliquency'].apply(lambda x: 1 if x>0 else 0)
    
    # Feature Generation
    df['Closed_Accounts'] = df['Total_Accounts']-df['Number_Open_Accounts']
    df['Monthly_Income'] = df['Annual_Income']/12
    df.drop("Months_Since_Deliquency",axis=1,inplace=True)
    df['Loan_Amount_As_Pct_Of_Income'] = np.round((df['Loan_Amount_Requested']/df['Annual_Income'])*100,2)

In [ ]:
def income_bracket(x):
    if x<=30000:
        return "Low"
    elif (x>30000 and x<=60000):
        return "Medium"
    elif (x>60000 and x<=90000):
        return "High"
    else:
        return "Very High"
def loan_amount_bracket(x):
    if x<=10000:
        return "Low"
    elif (x>10000 and x<=20000):
        return "Medium"
    elif (x>20000 and x<=30000):
        return "High"
    else:
        return "Very High"
    
for df in [train,test]:
    df['Income_Bracket'] = df['Annual_Income'].apply(income_bracket)
    df['Loan_Amount_Bracket'] = df['Loan_Amount_Requested'].apply(loan_amount_bracket)

In [ ]:
# Creating Groupby Features

In [ ]:
agg_func = {
    'Loan_Amount_Requested': ['min','max','mean','median','std']    
}
agg_func = train.groupby('Home_Owner').agg(agg_func)
agg_func.columns = [ 'Home_Owner_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
train = train.merge(agg_func, on=['Home_Owner'], how='left')

agg_func = {
    'Loan_Amount_Requested': ['min','max','mean','median','std']    
}
agg_func = test.groupby('Home_Owner').agg(agg_func)
agg_func.columns = [ 'Home_Owner_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
test = test.merge(agg_func, on=['Home_Owner'], how='left')

In [ ]:
agg_func = {
    'Loan_Amount_Requested': ['min','max','mean','median','std']    
}
agg_func = train.groupby('Purpose_Of_Loan').agg(agg_func)
agg_func.columns = [ 'Purpose_Of_Loan_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
train = train.merge(agg_func, on=['Purpose_Of_Loan'], how='left')

agg_func = {
    'Loan_Amount_Requested': ['min','max','mean','median','std']    
}
agg_func = test.groupby('Purpose_Of_Loan').agg(agg_func)
agg_func.columns = [ 'Purpose_Of_Loan_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
test = test.merge(agg_func, on=['Purpose_Of_Loan'], how='left')

In [ ]:
agg_func = {
    'Loan_Amount_Requested': ['min','max','mean','median','std']    
}
agg_func = train.groupby('Length_Employed').agg(agg_func)
agg_func.columns = [ 'Length_Employed_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
train = train.merge(agg_func, on=['Length_Employed'], how='left')

agg_func = {
    'Loan_Amount_Requested': ['min','max','mean','median','std']    
}
agg_func = test.groupby('Length_Employed').agg(agg_func)
agg_func.columns = [ 'Length_Employed_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
test = test.merge(agg_func, on=['Length_Employed'], how='left')

In [ ]:
agg_func = {
    'Annual_Income': ['min','max','mean','median','std']    
}
agg_func = train.groupby('Length_Employed').agg(agg_func)
agg_func.columns = [ 'Length_Employed_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
train = train.merge(agg_func, on=['Length_Employed'], how='left')

agg_func = {
    'Annual_Income': ['min','max','mean','median','std']    
}
agg_func = test.groupby('Length_Employed').agg(agg_func)
agg_func.columns = [ 'Length_Employed_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
test = test.merge(agg_func, on=['Length_Employed'], how='left')

In [ ]:
agg_func = {
    'Annual_Income': ['min','max','mean','median','std']    
}
agg_func = train.groupby('Purpose_Of_Loan').agg(agg_func)
agg_func.columns = [ 'Purpose_Of_Loan_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
train = train.merge(agg_func, on=['Purpose_Of_Loan'], how='left')

agg_func = {
    'Annual_Income': ['min','max','mean','median','std']    
}
agg_func = test.groupby('Purpose_Of_Loan').agg(agg_func)
agg_func.columns = [ 'Purpose_Of_Loan_' + ('_'.join(col).strip()) for col in agg_func.columns.values]
agg_func.reset_index(inplace=True)
test = test.merge(agg_func, on=['Purpose_Of_Loan'], how='left')

In [ ]:
cols = [col for col in train.columns if train[col].dtype=="object"]
for col in cols:
    train[col] = train[col].astype('category')
    train[col] = train[col].cat.codes
    
    test[col] = test[col].astype('category')
    test[col] = test[col].cat.codes
    
cols = ['Number_Open_Accounts','Total_Accounts','Closed_Accounts','Inquiries_Last_6Mo']
for item in cols:
    train[item] = train[item].astype('category')
    train[item] = train[item].cat.codes
    
    test[item] = test[item].astype('category')
    test[item] = test[item].cat.codes

In [ ]:
cols = [col for col in train.columns if col not in ['Length_Employed','Home_Owner','Income_Verified','Purpose_Of_Loan','Inquiries_Last_6Mo','Number_Open_Accounts','Total_Accounts','Gender','Interest_Rate','Deliquency_Status','Closed_Accounts','Income_Bracket','Loan_Amount_Bracket','Count_Of_Length_Of_Employment','Count_Of_Home_Owner','Count_Of_Income_Verified','Count_Of_Purpose_Of_Loan']]
for col in cols:
    train[[col]] = np.log1p(train[[col]])
    test[[col]] = np.log1p(test[[col]])

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
y = train['Interest_Rate']
X = train[[col for col in train.columns if col!="Interest_Rate"]]

In [ ]:
kf = StratifiedKFold(n_splits=5,shuffle=False,random_state=42)

In [ ]:
clf = lgb.LGBMClassifier(boosting_type='gbdt',objective='multiclass',num_class=3,random_state=42,n_jobs=-1,verbose=1,)
params = {"max_depth":[4,6,8,10,-1],
          "learning_rate":[0.01,0.03,0.05,0.1,0.3],
          "subsample":[0.5,0.6,0.7,0.8,0.9],
          "colsample_bytree":[0.5,0.6,0.7,0.8,0.9],
          "reg_alpha":[0,0.25,0.5,1,2],
          "reg_lambda":[0,0.25,0.5,1,2],
          "num_leaves":[7,15,31,63,127],
          "min_data_in_leaf":[1,3,5,7,10,25,50,75,100],
          "n_estimators":list(range(50,1001,100))}
random_search = RandomizedSearchCV(estimator=clf,param_distributions=params,cv=kf.split(X,y),scoring='f1_weighted')
random_search.fit(X,y)

In [ ]:
random_search.best_estimator_,random_search.best_params_,random_search.best_score_

In [ ]:
plt.figure(figsize=(18,6))
ser = pd.Series(random_search.best_estimator_.feature_importances_,X.columns).sort_values()
ser.plot(kind='bar')

In [ ]:
clf_1 = xgb.XGBClassifier(random_state=42,n_jobs=-1,verbosity=1,objective="multi:softmax",num_class=3)
params = {"max_depth":[4,6,8,10],
          "n_estimators":list(range(50,501,100)),
          "learning_rate":[0.01,0.05,0.1,0.3],
          "subsample":[0.5,0.6,0.7,0.8,0.9],
          "colsample_bytree":[0.5,0.6,0.7,0.8,0.9],
          "reg_alpha":[0.25,0.5,1],
          "reg_lambda":[0.25,0.5,1]}
random_search_1 = RandomizedSearchCV(estimator=clf_1,param_distributions=params,cv=kf.split(X,y),scoring='f1_weighted')
random_search_1.fit(X,y)

In [ ]:
random_search_1.best_estimator_,random_search_1.best_params_,random_search_1.best_score_

In [ ]:
plt.figure(figsize=(18,6))
ser = pd.Series(random_search_1.best_estimator_.feature_importances_,X.columns).sort_values()
ser.plot(kind='bar')

In [ ]:
Submission['Interest_Rate_LGB'] = random_search.best_estimator_.predict(test)
Submission['Interest_Rate_XGB'] = random_search_1.best_estimator_.predict(test)
#Submission.to_csv("Sub_78.csv",index=None)

In [ ]:
Submission['Interest_Rate'] = Submission[["Interest_Rate_LGB", "Interest_Rate_XGB"]].min(axis=1)

In [ ]:
Submission.drop(['Interest_Rate_LGB','Interest_Rate_XGB'],axis=1,inplace=True)
Submission.to_csv("Sol_14.csv",index=None)